In [1]:
pip install nba_api

     ------------------------------------ 125.6/125.6 kB 670.5 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 62.8/62.8 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 160.4/160.4 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB 1.7 MB/s eta 0:00:00
     -------------------------------------- 140.4/140.4 kB 1.7 MB/s eta 0:00:00
Using legacy 'setup.py install' for nba_api, since package 'wheel' is not installed.
  Running setup.py install for nba_api: started
  Running setup.py install for nba_api: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [3]:
gamefinder = leaguegamefinder.LeagueGameFinder(date_from_nullable='01/31/2020', league_id_nullable='00')
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42021,1610612738,BOS,Boston Celtics,0042100406,2022-06-16,BOS vs. GSW,L,239,90,...,0.917,11,30,41,27,8,8,22,16,-13.0
1,42021,1610612744,GSW,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,241,103,...,1.000,15,29,44,27,13,7,15,20,13.0
2,42021,1610612744,GSW,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,240,104,...,0.867,4,35,39,23,9,2,6,28,10.0
3,42021,1610612738,BOS,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,238,94,...,0.677,8,39,47,18,2,2,18,16,-10.0
4,42021,1610612738,BOS,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,241,97,...,0.737,11,31,42,22,8,6,15,17,-10.0


## Clean and Explore data

In [4]:
games.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [5]:
games = games[['TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'PLUS_MINUS']]

In [6]:
games

,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PLUS_MINUS
0,Boston Celtics,0042100406,2022-06-16,BOS vs. GSW,L,-13.0
1,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,13.0
2,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,10.0
3,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,-10.0
4,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,-10.0
...,...,...,...,...,...,...
6147,Denver Nuggets,0021900724,2020-01-31,DEN @ MIL,W,12.0
6148,Milwaukee Bucks,0021900724,2020-01-31,MIL vs. DEN,L,-12.0
6149,Portland Trail Blazers,0021900726,2020-01-31,POR @ LAL,W,8.0
6150,Los Angeles Lakers,0021900726,2020-01-31,LAL vs. POR,L,-8.0


final DataFrame: two columns: 1.result of game: target 2.score statistics comparing two teams: feature

In [7]:
import pandas as pd

In [8]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6152 entries, 0 to 6151
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TEAM_NAME   6152 non-null   object 
 1   GAME_ID     6152 non-null   object 
 2   GAME_DATE   6152 non-null   object 
 3   MATCHUP     6152 non-null   object 
 4   WL          6152 non-null   object 
 5   PLUS_MINUS  6152 non-null   float64
dtypes: float64(1), object(5)
memory usage: 288.5+ KB


In [9]:
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])

In [10]:
games['GAME_DATE']

0      2022-06-16
1      2022-06-16
2      2022-06-13
3      2022-06-13
4      2022-06-10
          ...    
6147   2020-01-31
6148   2020-01-31
6149   2020-01-31
6150   2020-01-31
6151   2020-01-31
Name: GAME_DATE, Length: 6152, dtype: datetime64[ns]

In [12]:
games = games.sort_values('GAME_DATE')
games

,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PLUS_MINUS
6151,New Orleans Pelicans,0021900723,2020-01-31,NOP vs. MEM,W,28.0
6147,Denver Nuggets,0021900724,2020-01-31,DEN @ MIL,W,12.0
6146,Phoenix Suns,0021900725,2020-01-31,PHX vs. OKC,L,-4.0
6148,Milwaukee Bucks,0021900724,2020-01-31,MIL vs. DEN,L,-12.0
6149,Portland Trail Blazers,0021900726,2020-01-31,POR @ LAL,W,8.0
...,...,...,...,...,...,...
4,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,-10.0
3,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,-10.0
2,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,10.0
1,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,13.0


In [14]:
games['avg_30_plus_minus'] = games.groupby('TEAM_NAME')['PLUS_MINUS'].transform(lambda x: x.rolling(30, closed='left').mean())


In [15]:
games

,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PLUS_MINUS,avg_30_plus_minus
6151,New Orleans Pelicans,0021900723,2020-01-31,NOP vs. MEM,W,28.0,NaN
6147,Denver Nuggets,0021900724,2020-01-31,DEN @ MIL,W,12.0,NaN
6146,Phoenix Suns,0021900725,2020-01-31,PHX vs. OKC,L,-4.0,NaN
6148,Milwaukee Bucks,0021900724,2020-01-31,MIL vs. DEN,L,-12.0,NaN
6149,Portland Trail Blazers,0021900726,2020-01-31,POR @ LAL,W,8.0,NaN
...,...,...,...,...,...,...,...
4,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,-10.0,8.366667
3,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,-10.0,7.100000
2,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,10.0,3.900000
1,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,13.0,3.766667


In [16]:
games[games['TEAM_NAME']=='Toronto Raptors'].head(35)

,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PLUS_MINUS,avg_30_plus_minus
6142,Toronto Raptors,0021900720,2020-01-31,TOR @ DET,W,13.0,NaN
6117,Toronto Raptors,0021900740,2020-02-02,TOR vs. CHI,W,27.0,NaN
6067,Toronto Raptors,0021900758,2020-02-05,TOR vs. IND,W,1.0,NaN
6052,Toronto Raptors,0021900772,2020-02-07,TOR @ IND,W,9.0,NaN
6023,Toronto Raptors,0021900781,2020-02-08,TOR vs. BKN,W,1.0,NaN
5992,Toronto Raptors,0021900796,2020-02-10,TOR vs. MIN,W,11.0,NaN
5971,Toronto Raptors,0021900809,2020-02-12,TOR @ BKN,L,-10.0,NaN
5923,Toronto Raptors,0021900829,2020-02-21,TOR vs. PHX,W,17.0,NaN
5899,Toronto Raptors,0021900843,2020-02-23,TOR vs. IND,W,46.0,NaN
5858,Toronto Raptors,0021900858,2020-02-25,TOR vs. MIL,L,-11.0,NaN


In [17]:
msk = games['MATCHUP'].str.contains('@')
games_away = games[msk]
games_home = games[~msk]

In [18]:
games_home.shape

(3076, 7)

In [19]:
games_away.shape

(3076, 7)

In [20]:
games_home

,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PLUS_MINUS,avg_30_plus_minus
6151,New Orleans Pelicans,0021900723,2020-01-31,NOP vs. MEM,W,28.0,NaN
6146,Phoenix Suns,0021900725,2020-01-31,PHX vs. OKC,L,-4.0,NaN
6148,Milwaukee Bucks,0021900724,2020-01-31,MIL vs. DEN,L,-12.0,NaN
6145,Houston Rockets,0021900722,2020-01-31,HOU vs. DAL,W,7.0,NaN
6150,Los Angeles Lakers,0021900726,2020-01-31,LAL vs. POR,L,-8.0,NaN
...,...,...,...,...,...,...,...
9,Golden State Warriors,0042100402,2022-06-05,GSW vs. BOS,W,19.0,2.533333
6,Boston Celtics,0042100403,2022-06-08,BOS vs. GSW,W,16.0,8.133333
4,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,-10.0,8.366667
2,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,10.0,3.900000


In [21]:
games_away

,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PLUS_MINUS,avg_30_plus_minus
6147,Denver Nuggets,0021900724,2020-01-31,DEN @ MIL,W,12.0,NaN
6149,Portland Trail Blazers,0021900726,2020-01-31,POR @ LAL,W,8.0,NaN
6144,Dallas Mavericks,0021900722,2020-01-31,DAL @ HOU,L,-7.0,NaN
6142,Toronto Raptors,0021900720,2020-01-31,TOR @ DET,W,13.0,NaN
6141,Chicago Bulls,0021900721,2020-01-31,CHI @ BKN,L,-15.0,NaN
...,...,...,...,...,...,...,...
8,Boston Celtics,0042100402,2022-06-05,BOS @ GSW,L,-19.0,9.433333
7,Golden State Warriors,0042100403,2022-06-08,GSW @ BOS,L,-16.0,3.900000
5,Golden State Warriors,0042100404,2022-06-10,GSW @ BOS,W,10.0,3.433333
3,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,-10.0,7.100000


In [22]:
games_merged = pd.merge(games_home, games_away, on='GAME_ID', suffixes=('_home', '_away'))
games_merged

,TEAM_NAME_home,GAME_ID,GAME_DATE_home,MATCHUP_home,WL_home,PLUS_MINUS_home,avg_30_plus_minus_home,TEAM_NAME_away,GAME_DATE_away,MATCHUP_away,WL_away,PLUS_MINUS_away,avg_30_plus_minus_away
0,New Orleans Pelicans,0021900723,2020-01-31,NOP vs. MEM,W,28.0,NaN,Memphis Grizzlies,2020-01-31,MEM @ NOP,L,-28.0,NaN
1,Phoenix Suns,0021900725,2020-01-31,PHX vs. OKC,L,-4.0,NaN,Oklahoma City Thunder,2020-01-31,OKC @ PHX,W,4.0,NaN
2,Milwaukee Bucks,0021900724,2020-01-31,MIL vs. DEN,L,-12.0,NaN,Denver Nuggets,2020-01-31,DEN @ MIL,W,12.0,NaN
3,Houston Rockets,0021900722,2020-01-31,HOU vs. DAL,W,7.0,NaN,Dallas Mavericks,2020-01-31,DAL @ HOU,L,-7.0,NaN
4,Los Angeles Lakers,0021900726,2020-01-31,LAL vs. POR,L,-8.0,NaN,Portland Trail Blazers,2020-01-31,POR @ LAL,W,8.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071,Golden State Warriors,0042100402,2022-06-05,GSW vs. BOS,W,19.0,2.533333,Boston Celtics,2022-06-05,BOS @ GSW,L,-19.0,9.433333
3072,Boston Celtics,0042100403,2022-06-08,BOS vs. GSW,W,16.0,8.133333,Golden State Warriors,2022-06-08,GSW @ BOS,L,-16.0,3.900000
3073,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,-10.0,8.366667,Golden State Warriors,2022-06-10,GSW @ BOS,W,10.0,3.433333
3074,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,10.0,3.900000,Boston Celtics,2022-06-13,BOS @ GSW,L,-10.0,7.100000


In [23]:
games_merged['avg_30_plus_minus_diff'] = games_merged['avg_30_plus_minus_home'] - games_merged['avg_30_plus_minus_away']


In [24]:
games_merged

,TEAM_NAME_home,GAME_ID,GAME_DATE_home,MATCHUP_home,WL_home,PLUS_MINUS_home,avg_30_plus_minus_home,TEAM_NAME_away,GAME_DATE_away,MATCHUP_away,WL_away,PLUS_MINUS_away,avg_30_plus_minus_away,avg_30_plus_minus_diff
0,New Orleans Pelicans,0021900723,2020-01-31,NOP vs. MEM,W,28.0,NaN,Memphis Grizzlies,2020-01-31,MEM @ NOP,L,-28.0,NaN,NaN
1,Phoenix Suns,0021900725,2020-01-31,PHX vs. OKC,L,-4.0,NaN,Oklahoma City Thunder,2020-01-31,OKC @ PHX,W,4.0,NaN,NaN
2,Milwaukee Bucks,0021900724,2020-01-31,MIL vs. DEN,L,-12.0,NaN,Denver Nuggets,2020-01-31,DEN @ MIL,W,12.0,NaN,NaN
3,Houston Rockets,0021900722,2020-01-31,HOU vs. DAL,W,7.0,NaN,Dallas Mavericks,2020-01-31,DAL @ HOU,L,-7.0,NaN,NaN
4,Los Angeles Lakers,0021900726,2020-01-31,LAL vs. POR,L,-8.0,NaN,Portland Trail Blazers,2020-01-31,POR @ LAL,W,8.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071,Golden State Warriors,0042100402,2022-06-05,GSW vs. BOS,W,19.0,2.533333,Boston Celtics,2022-06-05,BOS @ GSW,L,-19.0,9.433333,-6.900000
3072,Boston Celtics,0042100403,2022-06-08,BOS vs. GSW,W,16.0,8.133333,Golden State Warriors,2022-06-08,GSW @ BOS,L,-16.0,3.900000,4.233333
3073,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,-10.0,8.366667,Golden State Warriors,2022-06-10,GSW @ BOS,W,10.0,3.433333,4.933333
3074,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,10.0,3.900000,Boston Celtics,2022-06-13,BOS @ GSW,L,-10.0,7.100000,-3.200000


In [25]:
games_merged[['WL_home', 'avg_30_plus_minus_diff']]

,WL_home,avg_30_plus_minus_diff
0,W,NaN
1,L,NaN
2,L,NaN
3,W,NaN
4,L,NaN
...,...,...
3071,W,-6.900000
3072,W,4.233333
3073,L,4.933333
3074,W,-3.200000


In [26]:
games_model = games_merged[['WL_home', 'avg_30_plus_minus_diff']].dropna()
games_model

,WL_home,avg_30_plus_minus_diff
398,W,1.933333
399,L,-5.366667
401,L,-6.233333
402,W,2.466667
403,L,-8.066667
...,...,...
3071,W,-6.900000
3072,W,4.233333
3073,L,4.933333
3074,W,-3.200000


In [27]:
games_model['WL_home'] = games_model['WL_home'].map({'W': 1, 'L': 0})

In [28]:
games_model

,WL_home,avg_30_plus_minus_diff
398,1,1.933333
399,0,-5.366667
401,0,-6.233333
402,1,2.466667
403,0,-8.066667
...,...,...
3071,1,-6.900000
3072,1,4.233333
3073,0,4.933333
3074,1,-3.200000
